In [1]:
# importing libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import warnings
from IPython.core.interactiveshell import InteractiveShell
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'pandas'

## Data Explanation

Data source: https://archive.ics.uci.edu/ml/datasets/Estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition+

Data attributes:
The attributes of our observation data is: 
- Frequent consumption of high caloric food (FAVC).
- Frequency of consumption of vegetables (FCVC).
- Number of main meals (NCP).
- Consumption of food between meals (CAEC)
- Consumption of water daily (CH20)
- Consumption of alcohol (CALC)

The attributes related with the physical condition are: 
- Calories consumption monitoring (SCC).
- Physical activity frequency (FAF).
- Time using technology devices (TUE).
- Transportation used (MTRANS), 
- Gender
- Age
- Height
- Weight

### Reading in the data

In [ ]:
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'
InteractiveShell.ast_node_interactivity= 'all'
data = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
data.head(10)
data.tail(10)
#data.tail(10)
#data.info()

In [ ]:
#converting some of the features to round numbers as some of the variables were continuous and not round numbers
data.FCVC = data["FCVC"].round(decimals=0)
data.CH20 = data["CH2O"].round(decimals=0)
data.FAF= data["FAF"].round(decimals=0)
data.TUE= data["TUE"].round(decimals=0)
data.head(10)
data.tail(10)

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

# Summary statitics of the data

In [ ]:

data.describe().T
data.info()

In [ ]:
data.shape

In [ ]:
data['Gender'].value_counts()

In [ ]:
data['Age'].value_counts()

In [ ]:
data['family_history_with_overweight'].describe()

In [ ]:
data["Height"].describe()

In [ ]:
data['FAVC'].describe()
 

In [ ]:
data['NCP'].describe()

In [ ]:
data['CAEC'].describe()

In [ ]:
data['SMOKE'].describe()

In [ ]:
data['NObeyesdad'].describe()

In [ ]:
data['MTRANS'].describe()

In [ ]:
data['TUE'].describe()

In [ ]:
data['FAF'].describe()

In [ ]:
data['CH2O'].describe()

In [ ]:
data['SCC'].describe()

In [ ]:
data['CALC'].describe()

In [ ]:
data.sample(20)

# Data Visualization

### Analyzing the distribution of the gender of patients
From the analysis below, the observation data comprises of almost the same number of both male and female gender.

In [ ]:
#Analysing the distributin of the gender of respondent
sns.countplot(x = 'Gender', data = data)

In [ ]:
#Understanding how Calories consumption monitoring (SCC) influenced patients weight From the analysis below.

plt.bar('SCC', 'Weight', data = data)
plt.ylabel('Weight')
plt.xlabel('SCC')

In [ ]:
sns.kdeplot(x = 'Age', data = data)

In [ ]:
# Showing the distribution of the different levels of Obesity
_,ax = plt.subplots(figsize=(13,6))
sns.countplot(x = 'NObeyesdad', data = data, ax=ax)
pd.set_option('display.width',1000)
plt.show()


In [ ]:
#checkig for skewness 
pd.set_option("display.width", 1000)
data.hist(figsize=(10, 10))
plt.show()

In [ ]:
#checking for outliers

_, ax = plt.subplots(figsize=(15, 15))
data.plot(kind='box', ax= ax)

In [ ]:
#diagram showing that majority of the respondent were in the middle age btw 19-26, and 27-29
_,ax = plt.subplots(figsize=(13,6))
sns.scatterplot(x='NObeyesdad', y='Age', hue= 'Gender', ax=ax, data = data)

### Analyzing the gender with respect to their age distribution

In [ ]:
#Analysing gender with respect to their age distribution
sns.barplot(x = 'Gender', y = 'Age', data = data)

In [ ]:
#Analysing high caloric intaje with  with respect to their weight distribution
sns.violinplot(x = 'FAVC', y = 'Weight', data = data)

### Understanding the composition of patients with family history of overweight
From the analysis below we can deduce that the percentage of patients who have family history of being overweight is higer than the patients without family history of being overweight.

In [ ]:
#Understanding the composition of patients with family history of overweight
sns.countplot(x = 'family_history_with_overweight', data = data)

## Predictive Analytics/Machine learning

### Create the target variable from variable 'NObeyesdad"
Encoding the target variable

In [ ]:
#creating a function to help ascertain my data

def status_converter(value):
    if value == 'Normal_Weight':
        return 2
    elif value == 'Overweight_Level_I':
        return 3
    elif value == 'Overweight_Level_II':
        return 3
    elif value == 'Obesity_Type_I':
        return 4
    elif value == 'Obesity_Type_II':
        return 4
    elif value == 'Obesity_Type_III':
        return 4
    elif value == 'Insufficient_Weight':
        return 1
    else:
        return null

 

In [ ]:
#creating a new name for my target variable
data['ObesityStatus'] = data['NObeyesdad'].apply(status_converter)
data.head()

In [ ]:
corr_matrix = data.corr()
#plt.plot_size(10, 12)
sns.heatmap(corr_matrix, vmin=-1, vmax=1, annot=True)

### Visualizing the target variable

In [ ]:
sns.countplot(x = 'ObesityStatus', data = data)

In [ ]:
data.columns
#data_corr = data.corr() ['NObeyesdad'].abs().sort_values(ascending=False)
#print(data_corr)

In [ ]:
#Target variable
y = data.ObesityStatus

In [ ]:
#using the dummy encoding
categorical_variables = ['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','SCC','CALC','MTRANS']
data = pd.get_dummies(data, columns = categorical_variables)
data.head()

In [ ]:
y = data['ObesityStatus']
data.drop(columns = ['NObeyesdad', 'ObesityStatus'], inplace = True)
x = data

# We Convert the independent and target variables into a numpy array for PCA

x = x.values
y = y.values


In [ ]:
x

### Perform a train-test-split

In [ ]:
#from sklearn.model_selection import train_test_split # Import train_test_split function
#from sklearn.cross_validation import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) # 70% training and 30% test
print(y_test.shape)

In [ ]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.transform(X_test)
scaled_x_train



In [ ]:
components = None
pca = PCA(n_components=components)
pca.fit(scaled_x_train)
print('Variances(Percentage):')
print(pca.explained_variance_ratio_ * 100)



In [ ]:
#plotting a scree plot
components= len(pca.explained_variance_ratio_) \
    if components is None else components
plt.plot(range(1, components+1), np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel('Number of components')
plt.ylabel('Explained variance (%)')


In [ ]:
pca = PCA(n_components= 0.75)
pca.fit (scaled_x_train)

print('Cumulative Variances(Percentage) :')
print(np.cumsum(pca.explained_variance_ratio_ * 100))
components = len(pca.explained_variance_ratio_)
print(f'Number of components: (components)')

#making the skree plot
plt.plot(range(1, components + 1), np.cumsum(pca.explained_variance_ratio_ * 100))
plt.xlabel('Number of components')
plt.ylabel('Explained variance (%)')

In [ ]:
#understanding the importance of each features
#pca_components = abs(pca.components_)


In [ ]:
#understanding the importance of each features
pca_components = abs(pca.components_)
#printing the 7 most important features in each components
print('Top 7 most important features in each components')
for row in range(pca_components.shape[0]):
    temp = np.argpartition(-(pca_components[row]), 7)
    indices= temp[np.argsort((-pca_components[row]) [temp])] [:7]
    print(f'Components {row}: {data.columns[indices]. to_list() }')



In [ ]:
pca = PCA(n_components= components)
scaled_x_train = pca.fit_transform(scaled_x_train)
scaled_x_test = pca.fit_transform(scaled_x_test)


 Using Logistics Regression Model

In [ ]:
log_model = LogisticRegression(random_state=0)

# Training a logistics model
log_model.fit(scaled_x_train, y_train)
y_prediction = log_model.predict(scaled_x_test)
#checking for model accuracy
print(y_prediction)
accuracy = metrics.accuracy_score(y_test, y_prediction) * 100
accuracy
class_report = metrics.classification_report(y_test, y_prediction)
print('Report of Classification;\n', class_report)
#print(log_model.coef_)
#print(log_model.intercept_)
#confusion_matrix(y_test, y_prediction)


In [ ]:
#dt_model = DecisionTreeClassifier(max_depth=4, min_samples_split=2, criterion='gini', random_state=1)
dt_model = DecisionTreeClassifier()
# Training a decision tree model
dt_model.fit(scaled_x_train, y_train)
dt_y_prediction = dt_model.predict(scaled_x_test)
# Checking for model accuracy score
print(dt_y_prediction)
accuracy = metrics.accuracy_score(y_test, dt_y_prediction) * 100
accuracy

class_report = classification_report(y_test, y_prediction)
print('Report of Classification;\n', class_report)
confusion_matrix(y_test, y_prediction)
#tuning my model for a better performance
#SEED= 1
#dt_model= DecisionTreeClassifier(max_depth=4, min_samples_leaf=8, random_state= SEED)



Using a Support Vector Classifier

In [ ]:
svc_model = SVC()

# Training support vector classifier model
svc_model.fit(scaled_x_train, y_train)
svc_y_prediction = svc_model.predict(scaled_x_test)
#checking for model accuracy score
print(svc_y_prediction)
accuracy = metrics.accuracy_score(y_test, svc_y_prediction)*100
accuracy
class_report = classification_report(y_test, y_prediction)
print('Report of Classification;\n', class_report)



Using a Random Forest Classifier

In [ ]:
rfc_model = RandomForestClassifier()
#rfc_model = RandomForestClassifier(max_depth=4, min_samples_leaf=8)

# Training a random forest classifier model
rfc_model.fit(scaled_x_train, y_train)
rfc_y_prediction = rfc_model.predict(scaled_x_test)
# Checking for model accuracy score
print(rfc_y_prediction)
accuracy = metrics.accuracy_score(y_test, rfc_y_prediction) * 100
accuracy
confusion_matrix(y_test, y_prediction)
class_report = classification_report(y_test, y_prediction)
print('Report of Classification;\n', class_report)


#tuning my model for a better performance
#SEED= 1
#rfc_model= RandomForestClassifier(max_depth=4, min_samples_leaf=8, random_state= SEED)
